In [9]:
import os
import pickle as pkl
import pandas as pd
from collections import defaultdict
from datetime import datetime

def parse_and_fuse_timestamp(time_str, time_nano):
    # Parse the time string (e.g., "2024-04-17 09:19:41") and timeNano (e.g., "322223")
    dt = datetime.strptime(f"{time_str}_{time_nano.split('_')[-1]}", "%Y-%m-%d_%H_%M_%S_%f")
    return dt

def process_pkl_files(meta_dir, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Dictionary to store tracks, where each key is a Label and value is a list of detections
    tracks = defaultdict(list)

    # Iterate through all .pkl files in the Meta directory
    for pkl_file in os.listdir(meta_dir):
        if pkl_file.endswith('.pkl'):
            pkl_path = os.path.join(meta_dir, pkl_file)
            # Read the .pkl file
            with open(pkl_path, 'rb') as f:
                data = pkl.load(f)

            # Extract required fields
            label = data['Label']
            time_nano = data['timeNano']
            time = data['time']
            box = data['BOX']

            # Fuse time and timeNano for sorting
            fused_timestamp = parse_and_fuse_timestamp(time, time_nano)

            # Convert BOX [x1, y1, x2, y2] to four points [x1, y1, x2, y1, x2, y2, x1, y2]
            x1, y1, x2, y2 = [float(x) for x in box]
            box_points = [x1, y1, x2, y1, x2, y2, x1, y2]
            # print(label)
            # Append detection to the track for this Label
            tracks[label].append({
                'timestamp': fused_timestamp,
                'box_points': box_points
            })

    # Process each track (Label) and save its detections to a CSV file
    for label, detections in tracks.items():
        # Sort detections by timestamp to ensure chronological order within the track
        detections = sorted(detections, key=lambda x: x['timestamp'])

        # Prepare data for CSV: each row is a detection in the track
        csv_data = []
        for i, detection in enumerate(detections, 0):  # 0-based index for detections
            box_points = detection['box_points']
            # Pad box points with zeros to reach 32 values
            padded_points = box_points + [0.0] * (33 - len(box_points))
            # Create row: detection index + box points + final zero
            row = [i] + padded_points + [0.0]
            csv_data.append(row)

        # Create a DataFrame without headers for the track
        df = pd.DataFrame(csv_data)

        # Define CSV filename (e.g., track_001.csv)
        csv_filename = f"track_{str(label).zfill(3)}.csv"
        csv_path = os.path.join(output_dir, csv_filename)

        # Save the track's detections to CSV without headers
        df.to_csv(csv_path, index=False, header=False)
        print(f"Saved track for label {label} to {csv_path} with {len(detections)} detections")

def main():
    # Define paths
    meta_dir = '/home/pp/Downloads/2024_4_17/TPos/Meta'  # Path to Meta directory
    output_dir = './data/Tracks'  # Path to save CSV files

    # Process the .pkl files and save tracks
    process_pkl_files(meta_dir, output_dir)

if __name__ == '__main__':
    main()

Saved track for label 88399 to ./data/Tracks/track_88399.csv with 33 detections
Saved track for label 75704 to ./data/Tracks/track_75704.csv with 136 detections
Saved track for label 86629 to ./data/Tracks/track_86629.csv with 30 detections
Saved track for label 64495 to ./data/Tracks/track_64495.csv with 31 detections
Saved track for label 44885 to ./data/Tracks/track_44885.csv with 32 detections
Saved track for label 105113 to ./data/Tracks/track_105113.csv with 8 detections
Saved track for label 48064 to ./data/Tracks/track_48064.csv with 239 detections
Saved track for label 60272 to ./data/Tracks/track_60272.csv with 103 detections
Saved track for label 39755 to ./data/Tracks/track_39755.csv with 37 detections
Saved track for label 96458 to ./data/Tracks/track_96458.csv with 868 detections
Saved track for label 65066 to ./data/Tracks/track_65066.csv with 14 detections
Saved track for label 108239 to ./data/Tracks/track_108239.csv with 32 detections
Saved track for label 106340 to .